# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray
`

In [2]:
pip install pymongo

     ------------------------------------ 472.7/472.7 kB 519.1 kB/s eta 0:00:00
     ------------------------------------ 305.4/305.4 kB 370.4 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pprint

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
establishments_collection = db['establishments']
document = establishments_collection.find_one()
print(document)

{'_id': ObjectId('65bfe48aad8b4c18aca30707'), 'FHRSID': 254250, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000066174', 'BusinessName': 'The Pines Calyx', 'BusinessType': 'Other catering premises', 'BusinessTypeID': 7841, 'AddressLine1': 'The Pines Garden', 'AddressLine2': 'Beach Road', 'AddressLine3': 'St Margarets Bay', 'AddressLine4': 'Kent', 'PostCode': 'CT15 6DZ', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2021-08-17T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'}, 'RightToReply': '', 'Distance': 4587.362402580997, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'to

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BuisnessName": "Penang Flavours",
    "BuisnessType": "Restaurant/Cafe/Canteen",
    "BuisnessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "Longitude": "0.08384000",
        "Latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": "4623.9723280747176",
    "NewRatingPending": True
}

In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant_data)

In [11]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print("Penang Flavours has been added to the database.")
else:
    print("Failed to add Penang Flavours to the database.")

Penang Flavours has been added to the database.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1}

result = establishments.find_one(query, projection)
print(result)

{'_id': ObjectId('65bfe48aad8b4c18aca30708'), 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
new_restaurant_query = {"BusinessName": "Penang Flavours"}
new_restaurant_update = {"$set": {"BusinessTypeID": "1"}}

establishments.update_one(new_restaurant_query, new_restaurant_update)


UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [14]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one(new_restaurant_query)
print(updated_restaurant)

None


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

print(f"Number of documents deleted: {result.deleted_count}")

Number of documents deleted: 994


In [17]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

print(f"Number of documents with LocalAuthorityName 'Dover': {dover_count}")

Number of documents with LocalAuthorityName 'Dover': 0


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

if remaining_document:
    print("There are still documents remaining in the collection:")
    print(remaining_document)
else:
    print("No documents found in the collection after deletion.")

There are still documents remaining in the collection:
{'_id': ObjectId('65bfe48aad8b4c18aca309ee'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None,

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude


# Find documents with String latitude and longitude
documents_to_update = establishments.find({
    "$or": [
        {"Latitude": {"$type": "string"}},
        {"Longitude": {"$type": "string"}}
    ]
})

# Update documents with Decimal latitude and longitude
for document in documents_to_update:
    latitude = Decimal(document.get("Latitude", 0))
    longitude = Decimal(document.get("Longitude", 0))

    establishments.update_one(
        {"_id": document["_id"]},
        {
            "$set": {
                "Latitude": latitude,
                "Longitude": longitude
            }
        }
    )

print("Data type conversion complete.")


Data type conversion complete.


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue

establishments.update_many(
    {"RatingValue": {"$type": "string"}},
    [
        {
            '$set': {
                "RatingValue": {
                    "$toInt": "$RatingValue"
                }
            }
        }
    ]
)



UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Check that the coordinates and rating value are now numbers
cursor = establishments_collection.find({}, {"Latitude": 1, "Longitude": 1, "RatingValue": 1}).limit(5)
for document in cursor:
    print(document)

{'_id': ObjectId('65bfe48aad8b4c18aca309ee'), 'RatingValue': 5}
{'_id': ObjectId('65bfe48aad8b4c18aca309f1'), 'RatingValue': 4}
{'_id': ObjectId('65bfe48aad8b4c18aca309f2'), 'RatingValue': 5}
{'_id': ObjectId('65bfe48aad8b4c18aca309f3'), 'RatingValue': 5}
{'_id': ObjectId('65bfe48aad8b4c18aca309f4'), 'RatingValue': 5}
